In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Load data
df = pd.read_csv("/Users/mariahloehr/IICD/IICD/Data/top40_cell_cycle.csv")

# Combine 'M' and 'G2' into one class, if desired (optional)
df['phase'] = df['phase'].replace({'M': 'G2'})

# Define features and target
X = df.drop(columns=['phase', 'age'])
y = df['phase']

feature_names = X.columns.tolist()
X = X.to_numpy()
y = y.to_numpy()

# Split into training and testing set (optional, for evaluation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=949, stratify=y)

In [10]:
n_ratio = 0.2
N = len(X)

n = int(np.round(n_ratio * N))
r = np.random.RandomState()

## index of minipatch
idx_I = np.sort(r.choice(N, size=n, replace=False))
y_mp = y[np.ix_(idx_I)]
labels = list(set(y_mp))
print(labels)

['S', 'G2', 'G0', 'G1']


LOCO

In [3]:
import sys
sys.path.append("/Users/mariahloehr/IICD/IICD/feature_importance")

In [4]:
import locomp
from locomp import *
from locomp.MLmodels import *
import itertools
import importlib
from sklearn.base import BaseEstimator, RegressorMixin, clone
import itertools
from functools import partial
import multiprocessing as mp
import re

In [5]:
# define fit_func
def mlr(X,Y):
    mlr = LogisticRegression(
    #multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    C=1.0, 
    random_state=949
                    ).fit(X, Y)    
    return mlr

In [6]:
J1 = 0
J2 = 1
m_ratio = 0.2
n_ratio = 0.2
B = 100
fit_func = mlr

In [8]:
N=len(X_train)
M = len(X_train[0])
clas=np.unique(y_train)

N1 = len(X_train)
clas=set(y_train)

def buildMPClass(X,Y,n_ratio,m_ratio):
    N = len(X)
    M = len(X[0])
    n = int(np.ceil(n_ratio * N))
    m = int(np.ceil(m_ratio * M))
    r = np.random.RandomState()
    ## index of minipatch
    #3 stratified sampling
    Y_pd=pd.DataFrame(Y.reshape((len(Y),1)))

    idx_I =Y_pd.groupby(0, group_keys=False).apply(lambda x: x.sample(frac=n_ratio))
    idx_I = np.sort(list(idx_I.index)) # stratified sampling of subset of observations
    idx_F = np.sort(r.choice(M, size=m, replace=False)) # uniform sampling of subset of features
    ## record which obs/features are subsampled 
    x_mp=X[np.ix_(idx_I, idx_F)]
    y_mp=Y[np.ix_(idx_I)]
    return [idx_I,idx_F,x_mp,y_mp]

in_mp_obs,in_mp_feature = np.zeros((B,N),dtype=bool),np.zeros((B,M),dtype=bool)
predictions=[]
for b in range(B):
    [idx_I,idx_F,x_mp,y_mp] = buildMPClass(X_train,y_train,n_ratio,m_ratio)
    model = fit_func(x_mp,y_mp)
    prob = pd.DataFrame(model.predict_proba(X_train[:, idx_F]), columns=model.classes_)
    for i in (clas):
        if i not in prob.columns:
            prob[i]=0
        predictions.append(np.array(prob))
        in_mp_obs[b,idx_I]=True
        in_mp_feature[b,idx_F]=True  
    predict = [np.array(predictions),in_mp_obs,in_mp_feature]

[predictions,in_mp_obs,in_mp_feature]= predict
zeros=False

/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_83447/913891106.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idx_I =Y_pd.groupby(0, group_keys=False).apply(lambda x: x.sample(frac=n_ratio))
/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_83447/913891106.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idx_I =Y_pd.groupby(0, group_keys=False).apply(lambda x: x.sample(frac=n_ratio))
/v

In [7]:
x=LOCOMPClass(X_train,y_train,n_ratio,m_ratio,B,fit_func, selected_features=[],alpha=0.1,bonf=False)
x.run_loco()

/Users/mariahloehr/IICD/IICD/feature_importance/locomp/util_locomp.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idx_I =Y_pd.groupby(0, group_keys=False).apply(lambda x: x.sample(frac=n_ratio))
/Users/mariahloehr/IICD/IICD/feature_importance/locomp/util_locomp.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idx_I =Y_pd.groupby(0, group_keys=False).apply(lambda x: x.sample(frac=n_ratio))
/Users/mariahloehr

Error building with_j dataframe
clas: ['G0' 'G1' 'G2' 'S']
with_j example: [[7.37047900e-06 2.90990759e-05 9.45098968e-01 5.48645624e-02]
 [8.60567120e-08 1.63593499e-06 9.38693228e-01 6.13050497e-02]
 [3.97239863e-06 6.13138833e-05 9.53674439e-01 4.62602751e-02]
 [8.88841961e-05 7.83876135e-07 8.66055741e-01 1.33854591e-01]
 [1.00929644e-04 6.65108203e-04 1.68367869e-01 8.30866093e-01]
 [8.15771502e-02 1.79678975e-01 2.05767491e-01 5.32976385e-01]
 [6.02978242e-02 3.88932166e-02 4.42329606e-01 4.58479353e-01]
 [4.63504576e-04 2.79163277e-02 4.30190474e-01 5.41429694e-01]
 [3.39291775e-01 3.18772732e-01 3.40704575e-01 1.23091789e-03]
 [8.20524932e-03 1.86154309e-02 8.39472034e-01 1.33707286e-01]
 [5.04915321e-07 9.21254479e-06 9.77478884e-01 2.25113989e-02]
 [2.29326094e-02 2.31494909e-02 8.81543607e-01 7.23742927e-02]
 [7.42710220e-05 1.91030529e-02 9.67879495e-01 1.29431808e-02]
 [1.96798095e-06 1.81190270e-06 8.21041208e-01 1.78955013e-01]
 [2.84199125e-02 9.25252908e-03 7.36664229e

UnboundLocalError: cannot access local variable 'with_j' where it is not associated with a value